# Parsing PDFs Homework

With the power of pdfminer, pytesseract, Camelot, and Tika, let's analyze some documents!

> If at any point you think, **"I'm close enough, I'd just edit the rest of it in Excel"**: that's fine! Just make a note of it.

## A trick to use again and again

### Approach 1

Before we get started: when you want to take the first row of your data and set it as the header, use this trick.

In [1]:
import pandas as pd
from pdfminer.high_level import extract_text
import camelot
from tika import parser 
import numpy as np
import re

In [2]:
df = pd.DataFrame([
    [ 'fruit name', 'likes' ],
    [ 'apple', 15 ],
    [ 'carrot', 3 ],
    [ 'sweet potato', 45 ],
    [ 'peach', 12 ],
])
df

,0,1
0,fruit name,likes
1,apple,15
2,carrot,3
3,sweet potato,45
4,peach,12


In [3]:
# Set the first column as the columns
df.columns = df.loc[0]

# Drop the first row
df = df.drop(0)

df

,fruit name,likes
1,apple,15
2,carrot,3
3,sweet potato,45
4,peach,12


🚀 Done!

### Approach 2

Another alternative is to use `.rename` on your columns and just filter out the columns you aren't interested in. This can be useful if the column name shows up multiple times in your data for some reason or another.

In [4]:
# Starting with the same-ish data...
df = pd.DataFrame([
    [ 'fruit name', 'likes' ],
    [ 'apple', 15 ],
    [ 'carrot', 3 ],
    [ 'fruit name', 'likes' ],
    [ 'sweet potato', 45 ],
    [ 'peach', 12 ],
])
df

,0,1
0,fruit name,likes
1,apple,15
2,carrot,3
3,fruit name,likes
4,sweet potato,45
5,peach,12


In [5]:
df = df.rename(columns={
    0: 'fruit name',
    1: 'likes'
})
df = df[df['fruit name'] != 'fruit name']
df

,fruit name,likes
1,apple,15
2,carrot,3
4,sweet potato,45
5,peach,12


🚀 Done!

### Useful tips about coordinates

If you want to grab only a section of the page [Kull](https://jsoma.github.io/kull/#/) might be helpful in finding the coordinates.

> **Alternatively** run `%matplotlib notebook` in a cell. Afterwards, every time you use something like `camelot.plot(tables[0]).show()` it will get you nice zoomable, hoverable versions that include `x` and `y` coordinates as you move your mouse.

Coordinates are given as `"left_x,top_y,right_x,bottom_y"` with `(0,0)` being in the bottom left-hand corner.

Note that all coordinates are strings, for some reason. It won't be `[1, 2, 3, 4]` it will be `['1,2,3,4']`

# The homework

This is **mostly Camelot work**, because I don't really have any good image-based PDFs to stretch your wings on tesseract. If you know of any, let me know and I can put together another couple exercises.

## Prison Inmates

Working from [InmateList.pdf](InmateList.pdf), save a CSV file that includes every inmate.

* Make sure your rows are *all data*, and you don't have any people named "Inmate Name."


In [6]:
%matplotlib notebook

In [145]:
df = pd.read_csv('inmates.csv')

In [149]:
df[df['Inmate Name'] == 'Inmate Name']

,ICN #,Inmate Name,Facility,Booking Date


In [150]:
df_inmates = df['Inmate Name']

In [151]:
df_inmates.to_csv('Inmate Names.csv')

## WHO resolutions

Using [A74_R13-en.pdf](A74_R13-en.pdf), what ten member countries are given the highest assessments?

* You might need to have two separate queries, and combine the results: that last page is pretty awful!
* Always rename your columns
* Double-check that your sorting looks right......
* You can still get the answer even without perfectly clean data

In [8]:
data = camelot.read_pdf('A74_R13-en.pdf', pages='1,2,3,4,5,6', flavor='stream')

In [9]:
dataframe = []
for table in data:
    dataframe.append(table.df)

In [10]:
dataframe

[                                   0              1
 0                        Members and      WHO scale
 1                  Associate Members  for 2022–2023
 2                                                 %
 3                        Afghanistan         0.0070
 4                            Albania         0.0080
 5                            Algeria         0.1380
 6                            Andorra         0.0050
 7                             Angola         0.0100
 8                Antigua and Barbuda         0.0020
 9                          Argentina         0.9151
 10                           Armenia         0.0070
 11                         Australia         2.2101
 12                           Austria         0.6770
 13                        Azerbaijan         0.0490
 14                           Bahamas         0.0180
 15                           Bahrain         0.0500
 16                        Bangladesh         0.0100
 17                          Barbados         

In [11]:
df = pd.concat(dataframe).sort_index()
df_updated = df.drop(columns=[2,3,4]).rename(columns= {0: 'Members and Associate Members', 1: 'WHO Scale 2022-2023'})

In [12]:
df_countries = df_updated.replace(['Members and', 'WHO scale', 'Associate Members', 'for 2022–2023', '%', 'WHA74.13'], np.NaN).dropna()

In [13]:
df_countries

,Members and Associate Members,WHO Scale 2022-2023
3,Bosnia and Herzegovina,0.0120
3,Afghanistan,0.0070
3,Sao Tome and Principe,0.0010
3,Mauritania,0.0020
3,,
...,...,...
46,Marshall Islands,0.0010
46,France,4.4273
47,San Marino,0.0020
47,Gabon,0.0150


## The Avengers

Using [THE_AVENGERS.pdf](THE_AVENGERS.pdf), approximately how many lines does Captain America have as compared to Thor and Iron Man?

* Character names only: we're only counting `IRON MAN` as Iron Man, not `TONY`.
* Your new best friend might be `\n`
* Look up `.count` for strings

In [2]:
text_heroes = extract_text('THE_AVENGERS.pdf')
text_heroes

'Marvel’s THE AVENGERS\n\nWritten By\n\nJoss Whedon\n\nStory By\nZak Penn and Joss Whedon\n\nBased on the characters appearing in the comic books\nPublished by MARVEL Comics\n\nTranscribed to PDF using:\nCeltx Studio | 2012\n\nComposed In Celtx Studio\nWith the help of\nSiOmniaFicta’s Script\non fanfiction.net\n\nENTERTAINMENT PURPOSES ONLY\n\n\x0c1\n\n2\n\n3\n\n1\n\nEXT. UNKNOWN AREA OF SPACE\n\nSpace, a floating staircase among the rocks...THE OTHER\nspeaks to a DISEMBODIED FIGURE in a chair.\n\nTHE OTHER hands LOKI a scepter with a glowing blue gem...\n\nTHE OTHER (V.O)\n\nThe Tesseract has awakened. It is\non a little world, a human world.\nThey would wield its power.\n\nTHE OTHER (V.O)\nBut our ally knows its workings as\nthey never will. He is ready to\nlead. And our force...\n\nTHE OTHER (V.O)\n\n...our Chitauri, will follow. The\nworld will be his...the universe,\nyours. And the humans, what can\nthey do but burn?\n\nThe CHITAURI put on their face mask and get ready.\n\nA 3d mo

In [3]:
text_heroes.count('\nIRON MAN\n')

19

In [4]:
parsed_heroes = parser.from_file('THE_AVENGERS.pdf')

2021-07-24 13:34:31,351 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /var/folders/71/5014ff1x0zq2pr57709f1d6r0000gn/T/tika-server.jar.
2021-07-24 13:35:02,354 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [5]:
parsed_heroes

{'metadata': {'Content-Type': 'application/pdf',
  'Creation-Date': '2012-08-30T09:39:52Z',
  'Last-Modified': '2013-03-22T03:45:49Z',
  'Last-Save-Date': '2013-03-22T03:45:49Z',
  'PTEX.Fullbanner': 'This is pdfTeX using libpoppler, Version 3.141592-1.40.3-2.2 (Web2C 7.5.6) kpathsea version 3.5.6',
  'X-Parsed-By': ['org.apache.tika.parser.DefaultParser',
   'org.apache.tika.parser.pdf.PDFParser'],
  'X-TIKA:content_handler': 'ToTextContentHandler',
  'X-TIKA:embedded_depth': '0',
  'X-TIKA:parse_time_millis': '1101',
  'access_permission:assemble_document': 'true',
  'access_permission:can_modify': 'true',
  'access_permission:can_print': 'true',
  'access_permission:can_print_degraded': 'true',
  'access_permission:extract_content': 'true',
  'access_permission:extract_for_accessibility': 'true',
  'access_permission:fill_in_form': 'true',
  'access_permission:modify_annotations': 'true',
  'created': '2012-08-30T09:39:52Z',
  'date': '2013-03-22T03:45:49Z',
  'dc:format': 'applicat

In [6]:
parsed_heroes['content'].count('\nIRON MAN\n')

19

In [7]:
parsed_heroes['content'].replace('\n', '').count('CAPTAIN AMERICA')

40

In [8]:
parsed_heroes['content'].replace('\n', '').count('THOR')

49

## COVID data

Using [covidweekly2721.pdf](covidweekly2721.pdf), what's the total number of tests performed in Minnesota? Use the Laboratory Test Rates by County of Residence chart.

* You COULD pull both tables separately OR you could pull them both at once and split them in pandas.
* Remember you can do things like `df[['name','age']]` to ask for multiple columns

In [9]:
tables = camelot.read_pdf('covidweekly2721.pdf', pages='6')
lab_df = tables[1].df

In [10]:
lab_df

,0,1,2,3,4,5
0,County,Number of Tests,Cumulative Rate,County,Number of Tests,Cumulative Rate
1,Aitkin,"19,204","12,128",Martin,"39,876","19,974"
2,Anoka,"545,958","15,714",McLeod,"61,300","17,111"
3,Becker,"59,238","17,540",Meeker,"37,736","16,351"
4,Beltrami,"60,345","13,085",Mille Lacs,"44,688","17,369"
5,Benton,"77,865\n9,953","26,976",Morrison,"59,179","17,961"
6,Big Stone,"13,220","26,356",Mower,"79,446","20,061"
7,Blue Earth,"136,895","20,641",Murray,"15,175","18,167"
8,Brown,"55,709","22,097",Nicollet,"66,163","19,585"
9,Carlton,"79,882","22,477",Nobles,"30,979","14,185"


In [11]:
df = pd.DataFrame(lab_df)

In [12]:
df_1 = df[[0,1,2]]

df_2 = df[[3,4,5]]

In [13]:
df_1 = df_1.rename(columns={0: 'County', 1: 'Number of Tests', 2:'Cumulative Rate'}).drop(index=0)

In [14]:
df_2 = df_2.rename(columns={3: 'County', 4: 'Number of Tests', 5:'Cumulative Rate'}).drop(index=0)

In [15]:
df = pd.concat([df_1, df_2])

In [16]:
df.head(10)

,County,Number of Tests,Cumulative Rate
1,Aitkin,"19,204","12,128"
2,Anoka,"545,958","15,714"
3,Becker,"59,238","17,540"
4,Beltrami,"60,345","13,085"
5,Benton,"77,865\n9,953","26,976"
6,Big Stone,"13,220","26,356"
7,Blue Earth,"136,895","20,641"
8,Brown,"55,709","22,097"
9,Carlton,"79,882","22,477"
10,Carver,"154,305","15,367"


In [17]:
df[df['County'] == 'Benton']

,County,Number of Tests,Cumulative Rate
5,Benton,"77,865\n9,953","26,976"


In [18]:
df['Number of Tests'].str.contains('\n').value_counts()

False    87
True      1
Name: Number of Tests, dtype: int64

In [19]:
df['Number of Tests']= df['Number of Tests'].replace('77,865\n9,953', '77,865', regex=True)

In [20]:
df['Cumulative Rate'] = df['Cumulative Rate'].replace('26,976', '19,574', regex=True)

In [21]:
df['Number of Tests'].replace(',', '', regex=True).astype('int').sum()

10249823

## Theme Parks

Using [2019-Theme-Index-web-1.pdf](2019-Theme-Index-web-1.pdf), save a CSV of the top 10 theme park groups worldwide.

* You can clean the results or you can restrict the area the table is pulled from, up to you

In [152]:
table1 = camelot.read_pdf('2019-Theme-Index-web-1.pdf', pages='11', flavor='stream')

In [153]:
df_parks = pd.DataFrame(table1[0].df)
df_parks

,0,1,2,3,4
0,RANK\n1,GROUP NAME\nWALT DISNEY ATTRACTIONS,% CHANGE\n-0.8%,"ATTENDANCE\n2019\n 155,991,000","ATTENDANCE \n2018\n 157,311,000"
1,2,MERLIN ENTERTAINMENTS GROUP,0.9%,"67,000,000","66,400,000*"
2,3,OCT PARKS CHINA,9.4%,"53,970,000","49,350,000"
3,4,UNIVERSAL PARKS AND RESORTS,2.3%,"51,243,000","50,068,000"
4,5,FANTAWILD GROUP,19.8%,"50,393,000","42,074,000"
5,6,CHIMELONG GROUP,8.9%,"37,018,000","34,007,000"
6,7,SIX FLAGS INC.,2.5%,"32,811,000","32,024,000"
7,8,CEDAR FAIR ENTERTAINMENT COMPANY,7.8%,"27,938,000","25,912,000"
8,9,SEAWORLD PARKS & ENTERTAINMENT,0.2%,"22,624,000","22,582,000"
9,10,PARQUES REUNIDOS,6.2%,"22,195,000","20,900,000"


In [154]:
parse = df_parks[df_parks.index == 0]
parse

,0,1,2,3,4
0,RANK\n1,GROUP NAME\nWALT DISNEY ATTRACTIONS,% CHANGE\n-0.8%,"ATTENDANCE\n2019\n 155,991,000","ATTENDANCE \n2018\n 157,311,000"


In [157]:
df_parks.columns = df_parks.iloc[0].str.split('\n').str[0]
df_parks

,RANK,GROUP NAME,% CHANGE,ATTENDANCE,ATTENDANCE
0,RANK\n1,GROUP NAME\nWALT DISNEY ATTRACTIONS,% CHANGE\n-0.8%,"ATTENDANCE\n2019\n 155,991,000","ATTENDANCE \n2018\n 157,311,000"
1,2,MERLIN ENTERTAINMENTS GROUP,0.9%,"67,000,000","66,400,000*"
2,3,OCT PARKS CHINA,9.4%,"53,970,000","49,350,000"
3,4,UNIVERSAL PARKS AND RESORTS,2.3%,"51,243,000","50,068,000"
4,5,FANTAWILD GROUP,19.8%,"50,393,000","42,074,000"
5,6,CHIMELONG GROUP,8.9%,"37,018,000","34,007,000"
6,7,SIX FLAGS INC.,2.5%,"32,811,000","32,024,000"
7,8,CEDAR FAIR ENTERTAINMENT COMPANY,7.8%,"27,938,000","25,912,000"
8,9,SEAWORLD PARKS & ENTERTAINMENT,0.2%,"22,624,000","22,582,000"
9,10,PARQUES REUNIDOS,6.2%,"22,195,000","20,900,000"


In [158]:
parse = df_parks.loc[0].str.split('\n').str[1:3]
parse

0
RANK                                 [1]
GROUP NAME     [WALT DISNEY ATTRACTIONS]
% CHANGE                         [-0.8%]
ATTENDANCE          [2019,  155,991,000]
ATTENDANCE          [2018,  157,311,000]
Name: 0, dtype: object

In [163]:
df_parks = df_parks.drop([10, 11])

In [164]:
df_parks.to_csv('Top 10 Parks.csv')

## Hunting licenses

Using [US_Fish_and_Wildlife_Service_2021.pdf](US_Fish_and_Wildlife_Service_2021.pdf) and [a CSV of state populations](http://goodcsv.com/geography/us-states-territories/), find the states with the highest per-capita hunting license holders.

In [30]:
fish = camelot.read_pdf('US_Fish_and_Wildlife_Service_2021.pdf')
fish[0].df

,0,1,2,3,4,5,6,7
0,State,Paid Hunting License \nHolders*,"Resident Hunting Licenses,\nTags, Permits and ...","Non-Resident Hunting \nLicenses,\nTags, Permit...","Total Hunting License, \nTags,Permits & Stamps**","Cost - Resident Hunting \nLicenses,\nTags, Per...","Cost - Non-Resident Hunting \nLicenses,\nTags,...",Gross Cost - Hunting \nLicenses
1,AK,"93,559","423,501","59,235","482,736","$4,859,356","$9,046,715","$13,906,071"
2,AL,"452,400","601,683","45,397","647,080","$9,700,295","$6,715,734","$16,416,029"
3,AR,"343,300","349,098","150,728","499,826","$7,851,601","$11,271,653","$19,123,254"
4,AS,0,0,0,0,$0,$0,$0
5,AZ,"302,383","464,607","88,708","553,315","$13,931,397","$5,968,169","$19,899,566"
6,CA,"262,009","949,540","25,535","975,075","$20,270,941","$1,114,625","$21,385,566"
7,CO,"296,609","481,253","120,954","602,207","$13,270,492","$49,732,815","$63,003,307"
8,CT,"32,052","101,377","4,543","105,920","$1,855,046","$372,108","$2,227,154"
9,DC,0,0,0,0,$0,$0,$0


In [58]:
df_fish = pd.DataFrame(fish[0].df)
df_fish.columns = df_fish.iloc[0]
df_fish = df_fish.drop(0)

In [59]:
df_fish

,State,Paid Hunting License \nHolders*,"Resident Hunting Licenses,\nTags, Permits and Stamps","Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps","Total Hunting License, \nTags,Permits & Stamps**","Cost - Resident Hunting \nLicenses,\nTags, Permits and Stamps","Cost - Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps",Gross Cost - Hunting \nLicenses
1,AK,"93,559","423,501","59,235","482,736","$4,859,356","$9,046,715","$13,906,071"
2,AL,"452,400","601,683","45,397","647,080","$9,700,295","$6,715,734","$16,416,029"
3,AR,"343,300","349,098","150,728","499,826","$7,851,601","$11,271,653","$19,123,254"
4,AS,0,0,0,0,$0,$0,$0
5,AZ,"302,383","464,607","88,708","553,315","$13,931,397","$5,968,169","$19,899,566"
6,CA,"262,009","949,540","25,535","975,075","$20,270,941","$1,114,625","$21,385,566"
7,CO,"296,609","481,253","120,954","602,207","$13,270,492","$49,732,815","$63,003,307"
8,CT,"32,052","101,377","4,543","105,920","$1,855,046","$372,108","$2,227,154"
9,DC,0,0,0,0,$0,$0,$0
10,DE,"15,619","45,997","6,044","52,041","$899,857","$499,867","$1,399,724"


In [104]:
df_states = pd.read_csv('us-states-territories.csv')
df_states.drop(columns=['Type', 'Name'])
new_states = df_states.drop(columns=['Type', 'Name'])

In [114]:
df_states

,Type,Name,Abbreviation,Capital,Population (2015),Population (2019),area (square miles)
0,State,Alabama,AL,Montgomery,NaN,"4,903,185","52,420"
1,State,Alaska,AK,Juneau,NaN,"731,545","665,384"
2,State,Arizona,AZ,Phoenix,NaN,"7,278,717","113,990"
3,State,Arkansas,AR,Little Rock,NaN,"3,017,804","53,179"
4,State,California,CA,Sacramento,NaN,"39,512,223","163,695"
...,...,...,...,...,...,...,...
60,Territory,Kingman Reef,NaN,NaN,NaN,NaN,0.005
61,Territory,Midway Atoll,NaN,NaN,NaN,NaN,3
62,Territory,Navassa Island,NaN,NaN,NaN,NaN,3
63,Territory,Palmyra Atoll,NaN,NaN,NaN,NaN,1.5


In [143]:
merged = df_fish.merge(df_states, left_on='State', right_on='Abbreviation')
merged

,State,Paid Hunting License \nHolders*,"Resident Hunting Licenses,\nTags, Permits and Stamps","Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps","Total Hunting License, \nTags,Permits & Stamps**","Cost - Resident Hunting \nLicenses,\nTags, Permits and Stamps","Cost - Non-Resident Hunting \nLicenses,\nTags, Permits and Stamps",Gross Cost - Hunting \nLicenses,Type,Name,Abbreviation,Capital,Population (2015),Population (2019),area (square miles)
0,DC,0,0,0,0,$0,$0,$0,Federal District,District of Columbia,DC,NaN,NaN,"705,749",68
